# Splash dynamics on a cone surface



## Documentation and Imports


Created on 27-07-21

Author: Valentin Laplaud

Formulas and graphs describing the dynamics of a drop splashing on a cone

In [ ]:
# plotting stuff
import matplotlib as mpl
mpl.use('TkAgg')
%matplotlib inline

COLOR = 'white'
COLOR2 = 'black'

mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams['axes.edgecolor'] = COLOR

mpl.rcParams["figure.facecolor"] = COLOR2
mpl.rcParams["axes.facecolor"] = COLOR2
mpl.rcParams["savefig.facecolor"] = COLOR2

import matplotlib.pyplot as plt
from cycler import cycler


# numbers handling
import numpy as np
from numpy import pi
import pandas as pd

# signal processing 
from scipy.signal import savgol_filter, correlate, correlation_lags
from scipy.interpolate import interp1d


# to hide known warnings
import warnings
warnings.filterwarnings("ignore")

# General system functions
import os
import shutil
import sys

import time

# my functions
sys.path.append('../PythonFuncs/')
import VallapFunc_DP as vf
import DropGeometryClasses as dgc
import DropGeometryFuncs as dgf
import ImpactSimulationFuncs as isf


## Drop impact characteristics

### Top view schematics

In [ ]:
# Xc = 0
# Yc = 0
# Rc = 5

# Xd = 6
# Yd = 0
# Rd = 4

# r = vf.dist(Xd,Yd,Xc,Yc)


# tx = np.linspace(0,2*pi,100)

# fig,ax = plt.subplots(dpi=600)
# ax.set_aspect('equal', adjustable='box') 
# ax.axis('off')

# # Cone
# ax.plot(Xc,Yc,'ow',ms = 2)
# ax.plot(Xc + Rc*np.cos(tx),Yc + Rc*np.sin(tx), '-w')
# ax.plot([Xc, Xc + Rc*np.cos(tx[30])], [Yc, Yc + Rc*np.sin(tx[30])], '--w', lw = 0.5)
# ax.text(Xc + Rc*np.cos(tx[36])/2,Yc+ Rc*np.sin(tx[36])/2, 'Rc', c = 'w', size = Rc*2)


# # Drop
# ax.plot(Xd,Yd,'ob',ms = 2)
# ax.plot(Xd + Rd*np.cos(tx),Yd + Rd*np.sin(tx), '-b')
# ax.plot([Xd, Xd + Rd*np.cos(tx[90])], [Yd, Yd + Rd*np.sin(tx[90])], '--b', lw = 0.5)
# ax.text(Xd + Rd*np.cos(tx[93])/2,Yd + Rd*np.sin(tx[93])/2, 'Rd', c = 'b', size = Rd*2)

# # Geometry definition
# ax.plot([Xc, Xd], [Yc, Yd], '--r', lw = 0.5, zorder = -1)
# ax.text((Xc+Xd)/2,(Yc+Yd)/2+Rd/10,'r', c= 'r', size = 10);


### Adimensional numbers

In [ ]:
# ### Parameters
# V = 4 # [m/s] drop impact speed
# L = 0.003 # [m] cone diameter
# rho = 1000 # [kg/m3] water density
# mu = 0.001 # [Pa.s] water viscosity
# gamma = 0.07 # [N/m] water surface tension
# g = 9.8 # [m/s²] gravity constant

# ### Adimensional numbers

# # Reynolds (viscosity vs. intertia)
# Re = rho*V*L/mu
# print('Reynolds = ' + str(Re))

# # Weber (capilarity vs. inertia)
# We = rho*L*V**2/gamma
# print('Weber = ' + str(We))

# # Froude (gravity vs. intertia)
# Fr = V/np.sqrt(g*L)
# print('Froude = ' + str(Fr))

### Plots of fall probability


In [ ]:

# dropR = 3
# coneR = 4 # Cone radius
# Alpha = pi/4 # Cone vertical angle

# xcircle = coneR*np.cos(np.linspace(0,2*pi,100))
# ycircle = coneR*np.sin(np.linspace(0,2*pi,100))
# xgrid = np.arange(-(coneR+dropR), coneR+dropR, 0.1)
# ygrid = np.arange(-(coneR+dropR), coneR+dropR, 0.1)
# xx, yy = np.meshgrid(xgrid, ygrid)
# mask = np.sqrt(xx**2+yy**2)<(coneR+dropR)
# z = dgf.ProbaFall(np.sqrt(xx[mask]**2+yy[mask]**2),dropR,coneR)


# ### Drop-Cone distance
# rstep = (coneR+dropR)/98
# nstep = int(np.ceil((coneR+dropR)/rstep))
# r = np.linspace(rstep/2,coneR+dropR-3*rstep/2,nstep) # impact position
# Pr = dgf.ProbaFall(r,dropR,coneR)


# # probability of falling at a certain distance of the cone center
# fig10,ax10 = plt.subplots(dpi = 500)
# fig10.suptitle("Probability of falling at a distance 'r' from the center \n dropR = " + str(dropR) + " mm, coneR = " + str(coneR) + " mm. ")
# ax10.plot(r/(coneR+dropR)*100,Pr,'o', ms = 1)
# ax10.set_xlabel(' r  [% of (coneR+dropR)]');
# ax10.set_ylabel('Probability [%]');
# fig10.tight_layout()

# fig11, ax11 = plt.subplots(dpi = 500)
# ax11.set_aspect('equal', adjustable='box')
# ax11.set_xlabel('X')
# ax11.set_ylabel('Y')
# fig11.suptitle("Probability of falling at a distance 'r' from the center \n dropR = " + str(dropR) + " mm, coneR = " + str(coneR) + " mm. ")


# ax11.plot(xcircle,ycircle,'-w')
# sc11 = ax11.scatter(xx[mask],yy[mask],c=z,s=1,cmap='jet')
# fig11.colorbar(sc11, ax = ax11, label = 'proba (%)',shrink = 0.6)


## Simulations

In [ ]:
savepath = r'D:\Users\laplaud\Desktop\PostDoc\Data\SplashCups\Model\Images'

### Defining drops and cones

In [ ]:
# ## Definig diverse drops and cones

# D0 = dgc.Drop(2.9,0.2,30,4)
# D1 = dgc.Drop(2.9,1.5,30,4)
# D2 = dgc.Drop(2.9,2.5,30,4)
# D3 = dgc.Drop(2.9,3.5,30,4)
# D4 = dgc.Drop(2.9,4.5,30,4)
# D5 = dgc.Drop(2.9,5.5,30,4)
# D6 = dgc.Drop(2.9,6.5,30,4)
# D7 = dgc.Drop(2.9,7.5,30,4)

# P = dgc.Cone(2.7,np.pi/2) # flat pillar

# # Experimental cones
# C45_27 = dgc.Cone(2.7,np.pi/4) 
# C45_35 = dgc.Cone(3.5,np.pi/4) 
# C45_54 = dgc.Cone(5.4,np.pi/4) 

# C60 = dgc.Cone(5,np.pi/3)
# C30 = dgc.Cone(5,np.pi/6)



### Computing impacts

In [ ]:

# I_D0_C45 = C45_35.impact(D0)
# I_D4_C45 = C45_27.impact(D2)


# I_D4_P = P.impact(D4)
# I_D1_C30 = C30.impact(D1)
# I_D1_C60 = C60.impact(D1)


# I_D2_C45 = C45_27.impact(D2)
# I_D3_C45 = C45_27.impact(D3)
# I_D4_C45 = C45_27.impact(D4)
# I_D5_C45 = C45_27.impact(D5)
# I_D6_C45 = C45_27.impact(D6)
# I_D7_C45 = C45_27.impact(D7)


### Plots

#### Impact visualizations

In [ ]:

# dgc.Cone(3,np.pi/6).impact(dgc.Drop(2.9,3.5,50,4)).plot_3Dview()

# # C45_27.draw(nolabels=True,dropview = 'impact',drop=D3,title='45° cone + D1')
# # I_D1_C45.plot_JetFrac(title='Drop fraction in the jet, cone of 45°',nolabels=True)


# I_D1_C45.plot_splash_traj(np.linspace(0,3.5,50),nolabels=True,title='Cone of 45°')

# I_D2_C45.plot_splash_traj(np.linspace(0,3,50),nolabels=True,title='Cone of 45°')

# I_D3_C45.plot_splash_traj(np.linspace(0,2.5,50),nolabels=True,title='Cone of 45°')

# I_D4_C45.plot_splash_traj(np.linspace(0,2,50),nolabels=True,title='Cone of 45°')

# # I_D5_C45.plot_splash_traj(np.linspace(0,3,3),nolabels=True,title='Cone of 45°')

# # I_D6_C45.plot_splash_traj(np.linspace(0,3,3),nolabels=True,title='Cone of 45°')




#### Quantifications

In [ ]:
# Angle = np.pi/4 # [rad]
# npts = 20
# DropDiam = 3 # Diameter in [mm]
# ConeDiams = np.linspace(2,6,5) # Diameter in [mm]
# # ConeDiams = [3] # Diameter in [mm]


# isf.plotFracs(Angle,npts,DropDiam,ConeDiams)

In [ ]:
# Angle = np.pi/4 # [rad]
# npts = 50
# ConeDiam = 3 # Diameter in [mm]

# RelDropDiams = np.linspace(0.1,2,npts)

# RelOffCents = np.linspace(0.05,3,npts)

# f0,f1 = isf.PhaseDiagrams(Angle,ConeDiam,RelDropDiams,RelOffCents)

# f0.savefig(r'd:\Users\laplaud\Desktop\PostDoc\Code\DropProject_WithAna\Figures\PhsDgm_JetFrac.png')
# f1.savefig(r'd:\Users\laplaud\Desktop\PostDoc\Code\DropProject_WithAna\Figures\PhsDgm_VolRatio.png')

## Test Zone

### Ana's experimental data

In [ ]:
# data = pd.read_csv(r'd:\Users\laplaud\Desktop\PostDoc\Data\SplashCups\DataAna\all_angles.csv')

In [ ]:
# data = data.loc[(data['angle[degrees]']==45) &( data['IdealDiam[mm]']==2.7)]
# data

In [ ]:
# Rds = data['DropDiam'].values/2
# # Rcs = data['TargetDiam'].values
# OffCs = data['OffCentering'].values
# Names = data['Ind.1']

In [ ]:
# Drops = [dgc.Drop(Rd,OffC,35,5) for Rd,OffC in zip(Rds,OffCs)]
# C45 = dgc.Cone(2.7/2,np.pi/4)
# Impacts = [C45.impact(D) for D in Drops]

In [ ]:
# C45.impact(dgc.Drop(1.8,0.315,60,5)).plot_splash_traj(np.linspace(0,10,51),nolabels=True)

In [ ]:
# for t,i in zip(np.linspace(0,10,51),np.linspace(0,50,51)):
#     f,a = C45.impact(dgc.Drop(1.8,0.315,300,5)).plot_splash_traj([t],nolabels=True)
    
#     f.savefig(r'd:\Users\laplaud\Desktop\PostDoc\Code\DropProject_WithAna\TestFilm2\\' + str(i) + '.png' )
#     plt.close(f)

In [ ]:
# for I,name in zip(Impacts,Names):
#     I.plot_splash_traj(np.linspace(0,5,5),nolabels=False,title=name)

# Impacts[0].plot_splash_traj(np.linspace(0,4.5,15),nolabels=False)